In [2]:
pip install tensorflow

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached ml_dtypes-0.3.2-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
     -------------------------------------- 46.2/46.2 kB 765.3 kB/s eta 0:00:00
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached MarkupSafe-2.1.5-cp312-cp312-win_amd64.whl.metadata (3.1 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/377.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/377.1 MB 2.6 MB/s eta 0

In [1]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DitoVermelho\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DitoVermelho\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
data = {"intents": [
             {"tag": "greeting",
              "patterns": ["Oi", "Como vai você?", "E aí?", "Olá", "Você está bem?"],
              "responses": ["Olá!!", "Bem, e você?", "Tudo maravilha!", "Oi", "Muito melhor agora"],
             },
             {"tag": "age",
              "patterns": ["Você tem quantos anos?", "Quantos anos você tem?", "Quando é o seu aniversário?", "Quando você nasceu?"],
              "responses": ["Eu tenho 37 anos", "Eu nasci em 1983", "Meu aniversário é em dezembro", "05/12/1983"]
             },
             {"tag": "date",
              "patterns": ["Tem planos para o fim de semana?",
"O que você faz no tempo livre?", "Quais seus planos para este fim de semana?", "Vamos ao parque no fim de semana?"],
              "responses": ["Eu não tenho nada para fazer...", "Eu estou ocupado", "Pretendo dormir o tempo todo"]
             },
             {"tag": "name",
              "patterns": ["Qual o seu nome?", "Você tem apelido?", "Quem é você?", "Como você se chama?", "Quero saber seu nome"],
              "responses": ["Meu nome é Boop", "Sim, sou o Boop", "Boop."]
             },
             {"tag": "goodbye",
              "patterns": [ "Boa tarde", "Bom dia", "Boa noite", "tchau", "até mais", "obrigado", "adeus", "até"],
              "responses": ["Foi ótimo falar com você hoje", "Nos vemos depois", "Nos vemos em breve!"]
             }
]}

In [3]:
lemmatizer = WordNetLemmatizer()

In [4]:
words = []
classes = []
doc_X = []
doc_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # adicione a tag às classes se ainda não estiver lá
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# lematizar todas as palavras do vocabulário e convertê-las em minúsculas
# se as palavras não aparecem na pontuação
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

# classificando o vocabulário e as classes em ordem alfabética e tomando o 
# conjunto para garantir que não ocorram duplicatas
words = sorted(set(words))
classes = sorted(set(classes))

In [5]:
# lista de dados de treinamento
training = []
out_empty = [0] * len(classes)

# criando o modelo do saco de palavras
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)

# marca o índice da classe que o padrão atual está associado
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1

    # adicionar um BoW codificado a quente e classes associadas ao treinamento
    training.append([bow, output_row])

# embaralhar os dados e convertê-los em um array
random.shuffle(training)
training = np.array(training, dtype=object)

# divide os recursos e rótulos de destino
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [6]:
# definindo alguns parâmetros
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200

# o modelo de aprendizado profundo
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

d:\Documents\pos_Data Science\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\Documents\pos_Data Science\venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         6,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,365 (60.02 KB)

 Trainable params: 15,365 (60.02 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 840ms/step - accuracy: 0.2692 - loss: 1.5558
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4231 - loss: 1.5096
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.6154 - loss: 1.2923
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7692 - loss: 1.1653
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6154 - loss: 1.1389
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5385 - loss: 1.1948
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7308 - loss: 0.8372
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7692 - loss: 0.7358
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9231 - loss: 0.6098
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9615 - loss: 0.4877
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9231 - loss: 0.3687
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.884

In [7]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [8]:
# executando o chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Foi ótimo falar com você hoje
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Foi ótimo falar com você hoje
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Nos vemos depois
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Muito melhor agora
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Nos vemos depois
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Nos vemos em breve!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Nos vemos depois
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Nos vemos em breve!
